<a href="https://colab.research.google.com/github/lauravazqx/Metodos-Cuantitativos-en-Finanzas/blob/main/TareaExamen_MCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tarea Examen**
## **Métodos Cuantitativos en Finanzas 225-1**.
## **Árboles Binomiales y Simulación Monte Carlo**.



*   Gómez Arista Aimee Michelle
*   Salvador Vázquez Laura Teresa



# Parte I. Árboles Binomiales

Implementaciones en Python

## 1. c)
Crea una función en Python para
construir el árbol binomial de precios de la acción y calcular el
precio de la opción.


In [1]:
import numpy as np

def binomial_option_pricing(S0, K, r, T, dt, u, d, option_type="call"):
    """
    Calcula el precio de una opción europea utilizando el modelo binomial.

    Parámetros:
    - S0: Precio inicial de la acción.
    - K: Precio de ejercicio de la opción.
    - r: Tasa de interés libre de riesgo (anual).
    - T: Tiempo hasta el vencimiento (en años).
    - dt: Duración de cada periodo.
    - u: Factor de subida en cada periodo.
    - d: Factor de bajada en cada periodo.
    - option_type: Tipo de opción ("call" o "put").

    Retorna:
    - Precio de la opción al tiempo 0.
    """
    # Número de periodos
    N = int(T / dt)

    # Probabilidades neutrales al riesgo
    R = np.exp(r * dt)
    p = (R - d) / (u - d)

    # Construcción del árbol de precios de la acción
    prices = np.zeros((N + 1, N + 1))  # Matriz para los precios de la acción
    prices[0, 0] = S0
    for t in range(1, N + 1):
        for j in range(t + 1):
            prices[j, t] = S0 * (u**(t - j)) * (d**j)

    # Valores de la opción al vencimiento
    option_values = np.zeros((N + 1, N + 1))
    for j in range(N + 1):
        if option_type == "call":
            option_values[j, N] = max(0, prices[j, N] - K)
        elif option_type == "put":
            option_values[j, N] = max(0, K - prices[j, N])

    # Retroceso a través del árbol para calcular el precio de la opción
    for t in range(N - 1, -1, -1):
        for j in range(t + 1):
            option_values[j, t] = np.exp(-r * dt) * (
                p * option_values[j, t + 1] + (1 - p) * option_values[j + 1, t + 1]
            )

    # Precio de la opción al tiempo 0
    return option_values[0, 0]

# Ejemplo de uso
S0 = 50  # Precio inicial
K = 52  # Precio de ejercicio
r = 0.02  # Tasa libre de riesgo anual
T = 2  # Tiempo hasta el vencimiento en años
dt = 1  # Duración de cada periodo
u = 1.1  # Factor de subida
d = 0.9  # Factor de bajada

call_price = binomial_option_pricing(S0, K, r, T, dt, u, d, option_type="call")
call_price


2.9498896742597265

## 3. a)

Implementación en Python: Escribe una función en Python para construir el árbol binomial y calcular el precio de la opción put americana, considerando el ejercicio anticipado.

In [1]:
import numpy as np

def binomial_american_put(S0, K, r, u, d, T):
    """
    Calcula el precio de una opción put americana utilizando un modelo binomial.

    Parámetros:
    S0: Precio inicial de la acción.
    K: Precio de ejercicio de la opción.
    r: Tasa libre de riesgo (compuesta continuamente).
    u: Factor de subida.
    d: Factor de bajada.
    T: Número de periodos en el árbol.

    Retorna:
    - Precio de la opción en t=0.
    - Árbol de precios de la acción.
    - Árbol de valores de la opción.
    """
    # Probabilidad neutra al riesgo
    dt = 1  # Duración del periodo
    R = np.exp(r * dt)
    p = (R - d) / (u - d)

    # Construcción del árbol de precios
    prices = np.zeros((T + 1, T + 1))
    prices[0, 0] = S0

    for t in range(1, T + 1):
        for j in range(t + 1):
            prices[j, t] = S0 * (u ** (t - j)) * (d ** j)

    # Construcción del árbol de valores de la opción
    option_values = np.zeros((T + 1, T + 1))

    # Valores de la opción al vencimiento (t = T)
    for j in range(T + 1):
        option_values[j, T] = max(0, K - prices[j, T])

    # Retroceso a través del árbol para calcular el valor de la opción
    for t in range(T - 1, -1, -1):
        for j in range(t + 1):
            # Valor esperado descontado
            expected_value = np.exp(-r * dt) * (p * option_values[j, t + 1] + (1 - p) * option_values[j + 1, t + 1])
            # Valor de ejercicio anticipado
            option_values[j, t] = max(expected_value, K - prices[j, t])

    # El precio de la opción al inicio es el valor en el nodo raíz
    return option_values[0, 0], prices, option_values


# Ejemplo de uso
S0 = 40  # Precio inicial de la acción
K = 38  # Precio de ejercicio
r = 0.04  # Tasa libre de riesgo
u = 1.12  # Factor de subida
d = 0.88  # Factor de bajada
T = 3  # Número de periodos

# Calcular precio de la opción y mostrar árboles
precio_opcion, precios_arbol, valores_opcion = binomial_american_put(S0, K, r, u, d, T)

# Resultados
print(f"Precio de la opción put americana en t=0: {precio_opcion:.2f}\n")
print("Árbol de precios de la acción:")
print(precios_arbol)
print("\nÁrbol de valores de la opción:")
print(valores_opcion)


Precio de la opción put americana en t=0: 1.13

Árbol de precios de la acción:
[[40.      44.8     50.176   56.19712]
 [ 0.      35.2     39.424   44.15488]
 [ 0.       0.      30.976   34.69312]
 [ 0.       0.       0.      27.25888]]

Árbol de valores de la opción:
[[ 1.1338165   0.33234151  0.          0.        ]
 [ 0.          2.90162109  1.04833844  0.        ]
 [ 0.          0.          7.024       3.30688   ]
 [ 0.          0.          0.         10.74112   ]]


# Parte I. Simulaciones Monte Carlo.

Implementaciones en Python

## 1. a)
Escribe un código en Python que realice la simulación Monte Carlo para calcular el precio de la opción call.


In [3]:
import numpy as np

# Parámetros de la opción
S0 = 50  # Precio actual de la acción
K = 55   # Precio de ejercicio
r = 0.03  # Tasa de interés libre de riesgo
sigma = 0.25  # Volatilidad anual
T = 1    # Tiempo hasta el vencimiento (1 año)
n_simulaciones = 10000  # Número de simulaciones

# Simulación de Monte Carlo
np.random.seed(42)  # Para reproducibilidad
payoffs = []

for _ in range(n_simulaciones):
    # Simulación de un camino del precio de la acción
    Z = np.random.normal(0, 1)  # Variable aleatoria normal
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

    # Calcular el payoff de la opción call
    payoff = max(ST - K, 0)
    payoffs.append(payoff)

# Estimación del precio de la opción
precio_opcion = np.exp(-r * T) * np.mean(payoffs)

print(f"El precio estimado de la opción call es: ${precio_opcion:.2f}")


El precio estimado de la opción call es: $3.63


## 2. a)
 Utiliza tu código de Monte Carlo
 y modifica el número de iteraciones para observar y graficar la convergencia del valor de la opción en funció del número de simulaciones

In [4]:
import numpy as np

# Parámetros de la opción
S0 = 50  # Precio actual de la acción
K = 55   # Precio de ejercicio
r = 0.03  # Tasa de interés libre de riesgo
sigma = 0.25  # Volatilidad anual
T = 1    # Tiempo hasta el vencimiento (1 año)

# Función para realizar la simulación de Monte Carlo
def monte_carlo_call_option(S0, K, r, sigma, T, n_simulaciones):
    payoffs = []
    np.random.seed(42)
    for _ in range(n_simulaciones):
        Z = np.random.normal(0, 1)  # Variable aleatoria normal
        ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
        payoff = max(ST - K, 0)
        payoffs.append(payoff)

    precio_opcion = np.exp(-r * T) * np.mean(payoffs)
    return precio_opcion

# Realizamos simulaciones con diferentes números de iteraciones
n_simulaciones_list = [100, 1000, 10000]
resultados = {}

for n in n_simulaciones_list:
    precio = monte_carlo_call_option(S0, K, r, sigma, T, n)
    resultados[n] = precio

# Mostrar los resultados
for n, precio in resultados.items():
    print(f"Precio estimado de la opción con {n} simulaciones: ${precio:.2f}")


Precio estimado de la opción con 100 simulaciones: $2.45
Precio estimado de la opción con 1000 simulaciones: $3.64
Precio estimado de la opción con 10000 simulaciones: $3.63


## 3. a)
 Modifica el código de simulación
 Monte Carlo para incluir el pago de dividendos y calcula el precio
 de la opión put.

In [6]:
import numpy as np

# Parámetros
S0 = 50          # Precio inicial de la acción
K = 48           # Precio de ejercicio
r = 0.04         # Tasa libre de riesgo
q = 0.02         # Tasa de dividendos
sigma = 0.20     # Volatilidad
T = 1            # Tiempo al vencimiento (en años)
N = 1000000      # Número de simulaciones

# Simulación Monte Carlo
np.random.seed(42)  # Fijar semilla para reproducibilidad
Z = np.random.standard_normal(N)  # Generar números aleatorios normales

# Cálculo del precio de la opción
ST = S0 * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
payoff = np.maximum(K - ST, 0)  # Payoff de la opción put
precio_put = np.exp(-r * T) * np.mean(payoff)  # Valor presente esperado

print(f"El precio de la opción put es: {precio_put:.4f}")


El precio de la opción put es: 2.5229
